In [5]:
import numpy as np
import pandas as pd
import pickle
import requests
import tensorflow as tf
from transformers import TFAutoModel

In [ ]:
# url='https://kkb-production.jupyter-proxy.kaggle.net/k/144559851/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..DjJ5yTXYdGFJM6QHcQBhww.x-7HWEk4k4BbKpdm8BuyiyiACegF6K2oLM5gFojZi2FfyK6aNide90yDwYxvUHkayVH4l203ZK5Dw0YD1G0mDHzej3VPIABeZAnh3BkyW1khatPEILHcWNnfTdUAJ9tRKQMlEjjWBb4mZOOm-6eWvE9_Iln3ENjhFh7rlU5pAR5X6NiIO0FPI2RqAmZIPfJlKv5sFb8MJ5mYsee5cHvBhA.rxEbGir3ERdC5UkxhdRTNg/proxy/files/BiomedVLP-CXR-BERT-general-ts_inputs.pkl?_xsrf=2%7C41122b45%7C7e453d9ebd4c16961b05a8c35be21bf4%7C1693419082'
# r = requests.get(url, allow_redirects=True)

# open('x_test.pkl', 'wb').write(r.content)

In [4]:
# with open('', 'wb') as f:
#     pickle.dump(tr_inputs, f)

In [6]:
with open('/kaggle/working/x_test.pkl', 'rb') as f:
    x_test = pickle.load(f)
# with open('/kaggle/working/x_train.pkl', 'rb') as f:
#     x_tr = pickle.load(f)
# with open('/kaggle/working/x_val.pkl', 'rb') as f:
#     x_val = pickle.load(f)

In [7]:
with open('/kaggle/input/y-train-test-val/y_test/kaggle/working/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)
# with open('/kaggle/input/y-train-test-val/y_tr/kaggle/working/y_tr.pkl', 'rb') as f:
#     y_tr = pickle.load(f)
# with open('/kaggle/input/y-train-test-val/y_val/kaggle/working/y_val.pkl', 'rb') as f:
#     y_val = pickle.load(f)

In [5]:
tr_ds = tf.data.Dataset.from_tensor_slices((x_tr['input_ids'], x_tr['attention_mask'], y_tr))
val_ds = tf.data.Dataset.from_tensor_slices((x_val['input_ids'], x_val['attention_mask'], y_val))

In [6]:
def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

# then we use the dataset map method to apply this transformation
tr_ds = tr_ds.map(map_func)
val_ds = val_ds.map(map_func)

In [7]:
BATCH_SIZE = 64
tr_ds = tr_ds.batch(BATCH_SIZE, drop_remainder=True)
val_ds = val_ds.batch(BATCH_SIZE, drop_remainder=True)

In [7]:
bert = TFAutoModel.from_pretrained("bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12",from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'bert.embeddings.position_ids', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [10]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [16]:
InteractiveSession.close(session)

In [8]:
N_EPOCHS = 4
LR = 0.001
DECAY = 1e-06
NUM_LABELS = 10

In [9]:
# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')
bertmodel = bert.bert(input_ids, attention_mask=mask)[1]
drp = tf.keras.layers.Dropout(0.1)(bertmodel)
ln1 = tf.keras.layers.Dense(32, activation='linear')(drp)
bn1 = tf.keras.layers.BatchNormalization()(ln1)
rl1 = tf.keras.layers.Dense(64, activation='relu')(bn1)
ln2 = tf.keras.layers.Dense(32, activation='linear')(rl1)
bn1 = tf.keras.layers.BatchNormalization()(ln2)
rl2 = tf.keras.layers.Dense(64, activation='relu')(bn1)
y = tf.keras.layers.Dense(NUM_LABELS, activation='sigmoid', name='outputs')(rl2)

In [10]:
# initialize model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# freeze bert layer
model.layers[2].trainable = False

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [11]:
optimizer = tf.keras.optimizers.AdamW(learning_rate=LR,weight_decay=DECAY)

In [12]:
loss = tf.keras.losses.BinaryCrossentropy()
metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),tf.keras.metrics.CategoricalAccuracy()]

In [13]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [14]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS
)

Epoch 1/4
1151/1151 [==============================] - 1878s 2s/step - loss: 0.5548 - precision: 0.3627 - recall: 0.0169 - categorical_accuracy: 0.3159 - val_loss: 0.5477 - val_precision: 0.3333 - val_recall: 4.7015e-05 - val_categorical_accuracy: 0.3041
Epoch 2/4
1151/1151 [==============================] - 1823s 2s/step - loss: 0.5504 - precision: 0.4849 - recall: 0.0091 - categorical_accuracy: 0.3247 - val_loss: 0.5461 - val_precision: 0.6136 - val_recall: 0.0013 - val_categorical_accuracy: 0.3508
Epoch 3/4
1151/1151 [==============================] - 1864s 2s/step - loss: 0.5495 - precision: 0.4971 - recall: 0.0122 - categorical_accuracy: 0.3296 - val_loss: 0.5462 - val_precision: 0.5116 - val_recall: 0.0010 - val_categorical_accuracy: 0.3951
Epoch 4/4
1151/1151 [==============================] - 1864s 2s/step - loss: 0.5488 - precision: 0.5022 - recall: 0.0188 - categorical_accuracy: 0.3336 - val_loss: 0.5462 - val_precision: 0.2857 - val_recall: 9.4029e-05 - val_categorical_accur

In [15]:
model.save('blubert-4')

In [16]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS
)

Epoch 1/4
1151/1151 [==============================] - 1864s 2s/step - loss: 0.5484 - precision: 0.5177 - recall: 0.0215 - categorical_accuracy: 0.3354 - val_loss: 0.5439 - val_precision: 0.6096 - val_recall: 0.0072 - val_categorical_accuracy: 0.3201
Epoch 2/4
1151/1151 [==============================] - 1824s 2s/step - loss: 0.5483 - precision: 0.5197 - recall: 0.0203 - categorical_accuracy: 0.3377 - val_loss: 0.5450 - val_precision: 0.6093 - val_recall: 0.0087 - val_categorical_accuracy: 0.3313
Epoch 3/4
1151/1151 [==============================] - 1823s 2s/step - loss: 0.5481 - precision: 0.5260 - recall: 0.0248 - categorical_accuracy: 0.3369 - val_loss: 0.5443 - val_precision: 0.6209 - val_recall: 0.0062 - val_categorical_accuracy: 0.3665
Epoch 4/4
1151/1151 [==============================] - 1864s 2s/step - loss: 0.5478 - precision: 0.5225 - recall: 0.0263 - categorical_accuracy: 0.3401 - val_loss: 0.5443 - val_precision: 0.5978 - val_recall: 0.0154 - val_categorical_accuracy: 0.3

In [17]:
model.save('bluebert-8')

In [18]:
with open('bluebert-8-pkl', 'wb') as f:
    pickle.dump(model, f)

In [19]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS
)

Epoch 1/4
1151/1151 [==============================] - 1864s 2s/step - loss: 0.5476 - precision: 0.5249 - recall: 0.0248 - categorical_accuracy: 0.3408 - val_loss: 0.5434 - val_precision: 0.6220 - val_recall: 0.0120 - val_categorical_accuracy: 0.3511
Epoch 2/4
1151/1151 [==============================] - 1864s 2s/step - loss: 0.5476 - precision: 0.5249 - recall: 0.0241 - categorical_accuracy: 0.3406 - val_loss: 0.5442 - val_precision: 0.6168 - val_recall: 0.0048 - val_categorical_accuracy: 0.3554
Epoch 3/4
1151/1151 [==============================] - 1864s 2s/step - loss: 0.5473 - precision: 0.5319 - recall: 0.0264 - categorical_accuracy: 0.3437 - val_loss: 0.5438 - val_precision: 0.5986 - val_recall: 0.0241 - val_categorical_accuracy: 0.3792
Epoch 4/4
1151/1151 [==============================] - 1823s 2s/step - loss: 0.5473 - precision: 0.5378 - recall: 0.0260 - categorical_accuracy: 0.3422 - val_loss: 0.5429 - val_precision: 0.6519 - val_recall: 0.0131 - val_categorical_accuracy: 0.3

In [20]:
model.save('bluebert-12')

In [21]:
with open('bluebert-12-pkl', 'wb') as f:
    pickle.dump(model, f)

In [10]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS
)

Epoch 1/4
1151/1151 [==============================] - 1833s 2s/step - loss: 0.5472 - precision: 0.5278 - recall: 0.0257 - categorical_accuracy: 0.3427 - val_loss: 0.5432 - val_precision: 0.6174 - val_recall: 0.0130 - val_categorical_accuracy: 0.3488
Epoch 2/4
1151/1151 [==============================] - 1818s 2s/step - loss: 0.5469 - precision: 0.5372 - recall: 0.0281 - categorical_accuracy: 0.3439 - val_loss: 0.5426 - val_precision: 0.6252 - val_recall: 0.0154 - val_categorical_accuracy: 0.3441
Epoch 3/4
1151/1151 [==============================] - 1859s 2s/step - loss: 0.5471 - precision: 0.5325 - recall: 0.0277 - categorical_accuracy: 0.3442 - val_loss: 0.5427 - val_precision: 0.6128 - val_recall: 0.0139 - val_categorical_accuracy: 0.3137
Epoch 4/4
1151/1151 [==============================] - 1818s 2s/step - loss: 0.5470 - precision: 0.5353 - recall: 0.0270 - categorical_accuracy: 0.3455 - val_loss: 0.5425 - val_precision: 0.6108 - val_recall: 0.0139 - val_categorical_accuracy: 0.3

In [11]:
model.save('bluebert-16')
with open('bluebert-16-pkl', 'wb') as f:
    pickle.dump(model, f)

In [9]:
from tensorflow.keras.models import load_model
from transformers import TFBertModel
model = load_model('bluebert-16',custom_objects={"TFBertModel": TFBertModel})

In [ ]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS
)

Epoch 1/4
1151/1151 [==============================] - 1837s 2s/step - loss: 0.5469 - precision: 0.5319 - recall: 0.0273 - categorical_accuracy: 0.3417 - val_loss: 0.5433 - val_precision: 0.6250 - val_recall: 0.0106 - val_categorical_accuracy: 0.3375
Epoch 2/4
1151/1151 [==============================] - 1864s 2s/step - loss: 0.5469 - precision: 0.5378 - recall: 0.0277 - categorical_accuracy: 0.3476 - val_loss: 0.5420 - val_precision: 0.5742 - val_recall: 0.0268 - val_categorical_accuracy: 0.3000
Epoch 3/4
1151/1151 [==============================] - 1864s 2s/step - loss: 0.5468 - precision: 0.5312 - recall: 0.0270 - categorical_accuracy: 0.3435 - val_loss: 0.5427 - val_precision: 0.6414 - val_recall: 0.0135 - val_categorical_accuracy: 0.3339
Epoch 4/4
 798/1151 [===================>..........] - ETA: 8:29 - loss: 0.5468 - precision: 0.5353 - recall: 0.0292 - categorical_accuracy: 0.3452

In [ ]:
model.save('bluebert-20')
with open('bluebert-20-pkl', 'wb') as f:
    pickle.dump(model, f)

In [19]:
print(history.history['loss'])
print(history.history['val_loss'])

[0.5469217300415039, 0.5469194650650024, 0.5468319654464722, 0.5467244386672974]
[0.5432729125022888, 0.5419809222221375, 0.5426820516586304, 0.5421112775802612]


In [20]:
with open('/kaggle/working/x_test.pkl', 'rb') as f:
    x_test = pickle.load(f)
with open('/kaggle/input/y-train-test-val/y_test/kaggle/working/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

In [21]:
probes = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

640/640 [==============================] - 437s 677ms/step


In [37]:
from sklearn.metrics import roc_auc_score,hamming_loss,accuracy_score,precision_score,precision_recall_fscore_support,PrecisionRecallDisplay
print(roc_auc_score(y_test, probes, average=None))

[0.57019601 0.56751444 0.53461736 0.54361935 0.55615932 0.63897738
 0.55771997 0.63870736 0.59488967 0.5452883 ]


Prediction

In [8]:
from tensorflow.keras.models import load_model
from transformers import TFBertModel
model_4 = load_model('bluebert-16',custom_objects={"TFBertModel": TFBertModel})
# model_3 = load_model('bluebert-12',custom_objects={"TFBertModel": TFBertModel})
# model_2 = load_model('bluebert-8',custom_objects={"TFBertModel": TFBertModel})
# model_1 = load_model('blubert-4',custom_objects={"TFBertModel": TFBertModel})

In [9]:
# probes_1 = model_1.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
# probes_2 = model_2.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
# probes_3 = model_3.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
probes_4 = model_4.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

640/640 [==============================] - 437s 675ms/step


In [9]:
threshold  = np.arange(0.1,0.9,0.01)
threshold

array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ,
       0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31,
       0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42,
       0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53,
       0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64,
       0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75,
       0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86,
       0.87, 0.88, 0.89])

In [10]:
with open('16-predictions', 'wb') as f:
    pickle.dump(probes_4, f)

In [11]:
import numpy as np
from sklearn.metrics import roc_curve, precision_score, recall_score, f1_score

# Example predicted labels (probabilities)
predicted_labels = probes_4
# True labels
true_labels = y_test

# Determine threshold for each label
thresholds = []
for label_id in range(predicted_labels.shape[1]):
    # Get probabilities for the current label
    label_probs = predicted_labels[:, label_id]
    # Get true labels for the current label
    label_true = true_labels[:, label_id]
    # Calculate the false positive rate, true positive rate, and thresholds
    fpr, tpr, roc_thresholds = roc_curve(label_true, label_probs)
    # Calculate the distances from the top-left corner (0, 1)
    distances = np.sqrt(np.square(1 - tpr) + np.square(fpr))
    # Find the index of the threshold that minimizes the distance
    best_index = np.argmin(distances)
    # Get the corresponding threshold
    best_threshold = roc_thresholds[best_index]
    thresholds.append(best_threshold)
# Print the thresholds for each label
for label_id, threshold in enumerate(thresholds):
    print(f"Label {label_id + 1}: {threshold}")

Label 1: 0.421903520822525
Label 2: 0.4115627706050873
Label 3: 0.33502262830734253
Label 4: 0.28258219361305237
Label 5: 0.22734390199184418
Label 6: 0.21220369637012482
Label 7: 0.19085118174552917
Label 8: 0.1919291913509369
Label 9: 0.1449456363916397
Label 10: 0.14179493486881256


In [12]:
cols = ['I10', 'E785', 'Z87891', 'K219', 'F329', 'I2510', 'F419', 'N179',
       'Z794', 'E039']

In [13]:
binary_predictions = (predicted_labels >= thresholds).astype(int)

In [15]:
from sklearn.metrics import classification_report
print("Classification_report:")
print(classification_report(y_test,binary_predictions,target_names=cols,zero_division=0))

Classification_report:
              precision    recall  f1-score   support

         I10       0.48      0.52      0.50      8790
        E785       0.48      0.56      0.52      8848
      Z87891       0.39      0.53      0.45      7418
        K219       0.32      0.53      0.40      6204
        F329       0.26      0.57      0.35      4676
       I2510       0.31      0.62      0.41      4595
        F419       0.21      0.54      0.31      3859
        N179       0.25      0.63      0.36      3932
        Z794       0.19      0.60      0.29      3139
        E039       0.17      0.53      0.26      3018

   micro avg       0.31      0.56      0.40     54479
   macro avg       0.31      0.56      0.38     54479
weighted avg       0.35      0.56      0.42     54479
 samples avg       0.33      0.55      0.36     54479

